A. Read the  dataset from the following random_sampled_data.csv Links to an external site. and assign it to a Pandas DataFrame as you learned in tutorial Lab2-3.

In [419]:
# The following line will import KNeighborsClassifier "Class"
# KNeighborsClassifier is name of a "sklearn class" to perform "KNN Classification" 

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
# reading a CSV file directly from Web, and store it in a pandas DataFrame:
# "read_csv" is a pandas function to read csv files from web or local device:

#Random 1000 samples and save the csv file
# Set the total number of rows in the CSV file
#total_rows = 852950  # You should replace this with the actual total number of rows in your dataset
# Number of rows you want to read
#num_rows = 1000
# Include the first row and randomly sample the remaining rows
#random_rows = [0] + random.sample(range(1, total_rows), num_rows - 1)
# Read the CSV file for the specified random rows
#df = pd.read_csv('../Final project/Crime_Data_from_2020_to_Present.csv', skiprows=lambda x: x not in random_rows)
#df.to_csv("random_sampled_data.csv", index=False)

df = pd.read_csv("random_sampled_data.csv")

df_cleaned = df.dropna()


# Print the DataFrame



print(df)

         DR_NO               Date Rptd                DATE OCC  TIME OCC  \
0    200104875  01/12/2020 12:00:00 AM  01/12/2020 12:00:00 AM      1955   
1    200816285  11/11/2020 12:00:00 AM  11/10/2020 12:00:00 AM      1700   
2    200304982  01/17/2020 12:00:00 AM  01/17/2020 12:00:00 AM      1825   
3    200504796  01/19/2020 12:00:00 AM  01/19/2020 12:00:00 AM      1800   
4    200316294  08/25/2020 12:00:00 AM  08/25/2020 12:00:00 AM       700   
..         ...                     ...                     ...       ...   
994  230616403  10/12/2023 12:00:00 AM  10/11/2023 12:00:00 AM      1420   
995  231407108  03/16/2023 12:00:00 AM  03/15/2023 12:00:00 AM      2200   
996  231815151  08/18/2023 12:00:00 AM  08/17/2023 12:00:00 AM      2300   
997  230509664  06/07/2023 12:00:00 AM  06/07/2023 12:00:00 AM      1400   
998  231404405  01/12/2023 12:00:00 AM  01/12/2023 12:00:00 AM      2235   

     AREA  AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0       1    Central          17

##### B. Drop the fallowing columns

In [420]:
new_x = df

Drop_columns = [
    'DR_NO',
    'TIME OCC',
    'Rpt Dist No',
    'Part 1-2',
    'LOCATION',
    'AREA NAME',
    'Weapon Used Cd',
    'Premis Desc',
    'Status Desc',
    'Crm Cd 1',
    'Crm Cd 2',
    'Crm Cd 3',
    'Crm Cd 4',
    'Cross Street',
    'Date Rptd',
    'Mocodes',
    'Weapon Desc',
    'Status',
    'Premis Cd',
    'LAT',
    'LON'
    ]

new_x = new_x.drop(columns=Drop_columns)

for col in new_x.columns:
    print(col)

DATE OCC
AREA
Crm Cd
Crm Cd Desc
Vict Age
Vict Sex
Vict Descent


##### C. Now, we want to use the categorical features as well! To this end, we have to perform a feature engineering process called OneHotEncoding for the categorical features. To do this, each categorical feature should be replaced with dummy columns in the feature table (one column for each possible value of a categorical feature), and then encode it in a binary manner such that only one of the dummy columns can take “1” at a time (and zero for the rest).

In [421]:
# Creating the Feature Matrix for dataset:



# Function to map month to season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'


# Extracting the month from the 'Date' column and mapping it to seasons
#new_x['Month'] = pd.to_datetime(new_x['date_occ']).dt.month

new_x['Month'] = pd.to_datetime(new_x['DATE OCC']).dt.month
new_x['Day_of_week'] = pd.to_datetime(new_x['DATE OCC']).dt.day
new_x['Season'] = new_x['Month'].apply(get_season)



# Clean df of nah, Vict Sex =X , and age = 0 and reset index
new_x.dropna(inplace=True, subset= ['Vict Age','Vict Sex','Vict Descent'])
new_x = new_x[new_x['Vict Sex'] != 'X'].reset_index(drop=True)
new_x = new_x[new_x['Vict Descent'] != 'X'].reset_index(drop=True)
new_x = new_x[new_x['Vict Age'] !=  0].reset_index(drop=True)

#Add categorical features as 1 or 0 for seasons
new_x['Spring'] = np.where((new_x['Season'] == 'Spring'), 1,0)
new_x['Summer'] = np.where((new_x['Season'] == 'Summer'), 1,0)
new_x['Fall'] = np.where((new_x['Season'] == 'Fall'), 1,0)
new_x['Winter'] = np.where((new_x['Season'] == 'Winter'), 1,0)

# #Add categorical featrues as 1 or 0 for Vict Sex
# new_x['Male'] = np.where((new_x['Vict Sex'] == 'M'), 1,0)
# new_x['Female'] = np.where((new_x['Vict Sex'] == 'F'), 1,0)

# #Add categorical features as 1 or 0 for Vict Descent
df_encoded = pd.get_dummies(new_x, columns= ['Vict Descent','Day_of_week','Crm Cd Desc','AREA','Crm Cd'])




feature_cols_list = list(df_encoded.columns.values)

remove_cols = [
    'DATE OCC',
    'Vict Sex',
    'Month',
    'Season',
    'Crm Cd Desc',
    'Male',
    'Female',
    'LOCATION',
    'AREA',
    'Crm cd'
    ]


feature_cols = list(set(feature_cols_list).difference(remove_cols))



# use the above list to select the features from the original DataFrame
X = df_encoded[feature_cols]
# select a Series of labels (the last column) from the DataFrame
y = df_encoded['Vict Sex'] # this is the original categorical labels (the latest revision of sklearn accepts non-numerical labels)


for col in X.columns:
    print(col)


Crm Cd Desc_ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT
Crm Cd Desc_BURGLARY FROM VEHICLE
Crm Cd Desc_CRIMINAL THREATS - NO WEAPON DISPLAYED
AREA_11
Vict Age
AREA_10
Crm Cd Desc_BRANDISH WEAPON
Crm Cd Desc_THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)
Fall
Crm Cd Desc_DOCUMENT FORGERY / STOLEN FELONY
AREA_21
Day_of_week_25
Day_of_week_28
Crm Cd Desc_INTIMATE PARTNER - SIMPLE ASSAULT
Day_of_week_18
Crm Cd Desc_INTIMATE PARTNER - AGGRAVATED ASSAULT
Crm Cd_110
Crm Cd_814
AREA_17
Crm Cd_666
Vict Descent_A
Day_of_week_30
Crm Cd_434
Crm Cd_810
Day_of_week_3
Crm Cd Desc_RAPE, ATTEMPTED
Crm Cd Desc_ORAL COPULATION
Crm Cd_230
AREA_6
Crm Cd_762
AREA_5
AREA_14
AREA_4
Day_of_week_22
Crm Cd Desc_CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT
Crm Cd Desc_CHILD NEGLECT (SEE 300 W.I.C.)
Crm Cd Desc_ATTEMPTED ROBBERY
Day_of_week_10
Vict Descent_O
Crm Cd_930
Crm Cd Desc_VEHICLE - ATTEMPT STOLEN
AREA_19
Crm Cd_480
Crm Cd Desc_DISTURBING THE PEACE
Crm Cd_341
Crm Cd Desc_LETTERS, LEWD  -  TELEPHONE CALLS

#### D.Split the dataset into testing and training sets with the following parameters: test_size=0.4, random_state=10

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(438, 183)
(293, 183)
(438,)
(293,)


##### F. Now, normalize (scale) the features. To normalize the data, you can simply use StandardScaler  from sklearn. (note: don’t normalize the target!). Remember that we can only use the statistics of X_train for normalization, and then apply it on both X_train and X_test:

In [423]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

#### E.Instantiate a KNN object with K= 59, train it on the training set and test it on the testing set. Then, calculate the accuracy of your prediction as you learned in Lab3.

In [424]:

k = 59

my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train_normalized, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test_normalized)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)


0.6006825938566553


#### F.Repeat part (E) for 1 to 60.

In [426]:

#K_values = [1,3,5,11,15,27,59]

K_values = list(range(1,60))
result_list = []

for K in K_values:

    my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=K, weights='uniform') # name of the object is arbitrary!
    my_knn_for_cs4661.fit(X.values, y)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10) # We can fix the random_state for reproducibility!

### Training ONLY on the training set:
# Training ONLY on the training set:

    X_train_normalized = scaler.fit_transform(X_train)
    X_test_normalized = scaler.transform(X_test)

    my_knn_for_cs4661.fit(X_train_normalized, y_train)

### Testing on the testing set:
# Testing on the testing set:

    y_predict = my_knn_for_cs4661.predict(X_test_normalized)


    accuracy = accuracy_score(y_test, y_predict)
    result_list.append(accuracy)
print(result_list)

[0.5392491467576792, 0.5221843003412969, 0.5563139931740614, 0.5460750853242321, 0.5699658703071673, 0.5426621160409556, 0.552901023890785, 0.5358361774744027, 0.5665529010238908, 0.5494880546075085, 0.5597269624573379, 0.5733788395904437, 0.5699658703071673, 0.590443686006826, 0.552901023890785, 0.5597269624573379, 0.5631399317406144, 0.5631399317406144, 0.5665529010238908, 0.5665529010238908, 0.552901023890785, 0.5426621160409556, 0.5665529010238908, 0.5665529010238908, 0.5631399317406144, 0.5733788395904437, 0.552901023890785, 0.5631399317406144, 0.5563139931740614, 0.5938566552901023, 0.5597269624573379, 0.5767918088737202, 0.5802047781569966, 0.5836177474402731, 0.5836177474402731, 0.590443686006826, 0.5870307167235495, 0.5631399317406144, 0.5767918088737202, 0.5699658703071673, 0.5631399317406144, 0.5733788395904437, 0.5733788395904437, 0.5631399317406144, 0.5665529010238908, 0.5597269624573379, 0.5767918088737202, 0.5631399317406144, 0.5733788395904437, 0.5631399317406144, 0.566

#### The best K value is 59